In [ ]:
# default_exp graphql.utilities

# GraphQL Utilities

> API details.

These are the tools I can re-use for all sorts of GraphQL interactions:

* dependencies
* functions
* conventions

Some of these will eventually make it to a more-general place, like just `utilities` instead of `graphql.utilities`.

In [ ]:
#hide
from nbdev.showdoc import *

Choosing a GraphQL client shouldn't be very important. There are [several](https://graphql.org/code/#python). I've decided to use [python-graphql-client](https://github.com/prisma-labs/python-graphql-client).

In [ ]:
import inspect
import dateutil
from datetime import datetime
from functools import partial

from python_graphql_client import GraphqlClient
import attr

## Utilities

### Memoize

I really like this concept, but I haven't found a convenient way to do it in Python. The idea is:

* run some functions only once, so it's reliable and efficient
* lazy-run these operations, so I don't have to think as much about how the code will be run
* separate concerns into layers, pushing details deeper

### Filter Parameters

This is a simple idea that makes the GraphQL variables easier to filter. It could be used in memoize, but isn't (yet).

In [ ]:
def memoize(self, key, default=None):
    hidden = f"_{key}"
    if not hasattr(self, hidden):
        if key in self.kwargs:
            setattr(self, hidden, self.kwargs[key])
        elif callable(default):
            try:
                setattr(self, hidden, default())
            except:
                setattr(self, hidden, default(self))
        else:
            setattr(self, hidden, default)
    return getattr(self, hidden)

def filter_parameters(fn, kwargs):
    parameters = inspect.signature(fn).parameters
    return {k:v for k, v in kwargs.items() if k in parameters}

In [ ]:
def fn(a=42):
    return a
class Context:
    def __init__(self, **kwargs):
        self.kwargs = kwargs
context = Context()

assert memoize(context, 'foo', fn) == 42
assert memoize(context, 'bar', 42) == 42

kwargs = dict(a=1, b=2)
kwargs = filter_parameters(fn, kwargs)
assert 'b' not in kwargs

del Context

### Utilities Feedback

So, this is the least I can do.

**Memoize** relies on kwargs saved to a class. It sets an attribute with the value from either kwargs or a default. The default can be a function. The function can either take no parameters or take the instance. This isn't very smart. It could be expanded with **filter_parameters**, but isn't.

## Casting

Sometimes I want something close to `str`, `float`, or `int`, but allowing for None values as well. These are simple but useful.

In [ ]:
def cast_to_non_empty_string(o):
    assert o is not None, "Provide a value."
    o = str(o)
    assert len(o) > 0, "Provide a non-empty string"
    return o

def cast_to_string(o, default=None):
    if o is None: return default
    return str(o)

def cast_to_datetime(o, default=None):
    if isinstance(o, datetime): return o
    try:
        return dateutil.parser.parse(o)
    except:
        return default
    
def cast_to_integer(o, default=0):
    if o is None: return default
    return int(o)

cast_to_integer_or_none = partial(cast_to_integer, default=None)

def cast_to_float(o, default=0.0):
    if o is None: return default
    return float(o)

cast_to_float_or_none = partial(cast_to_float, default=None)

def cast_to_time(o, default=None):
    if o is None: return default
    try:
        return dateutil.parser.parse(o).time()
    except:
        return default

In [ ]:
assert cast_to_non_empty_string('a') == 'a'

assert cast_to_string(None) is None
assert cast_to_string(1) == '1'

now = datetime.today()
assert cast_to_datetime(None) is None
assert cast_to_datetime(str(now)).date() == now.date()
assert cast_to_datetime('not going to work', default=now) == now

assert cast_to_integer(None) == 0
assert cast_to_integer('1') == 1
assert cast_to_integer(None, default=None) is None
assert cast_to_integer_or_none(None) is None

assert cast_to_float(None) == 0.0
assert cast_to_float('1') == 1.0
assert cast_to_float(None, default=None) is None
assert cast_to_float_or_none(None) is None

assert cast_to_time(None) is None
assert cast_to_time(str(now)) == now.time()
assert cast_to_time('not going to work') is None

### Casting Feedback

This is pretty straightforward. It's useful when assigning attributes in an `attrs` class.